In [1]:
import os
import random
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers, models, losses, optimizers, metrics

from model_train import load_c3d_model, train_msupcl_model, linear_evaluation
from data_uniform_sup import VideoDataGenerator
from paired_generator import PairedDataGenerator

In [2]:
seed = 2042
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)
input_shape = (16, 112, 112, 3)  # As defined in data generator
num_classes = 2  # Harmful or Safe
feature_dim = 512
num_epochs = 5
batch_size = 4

In [3]:
# Define dataset paths
violence_negative_dir = './data/violence_dataset/NonViolence'
violence_positive_dir = './data/violence_dataset/Violence'
tiktok_negative_dir = './data/tiktok/train/Safe'
tiktok_positive_dir = './data/tiktok/train/Harmful Content'


In [4]:
def sample_videos(directory, num_samples=100):
    all_videos = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.mp4')]
    sampled_videos = random.sample(all_videos, min(num_samples, len(all_videos)))
    return sampled_videos


In [5]:
# Violence dataset
violence_negative_videos = sample_videos(violence_negative_dir, 100)
violence_positive_videos = sample_videos(violence_positive_dir, 100)

# TikTok dataset
tiktok_negative_videos = sample_videos(tiktok_negative_dir, 100)
tiktok_positive_videos = sample_videos(tiktok_positive_dir, 100)


In [6]:
def split_data(negative_videos, positive_videos, train_ratio=0.55, val_ratio=0.15):
    # Combine and shuffle
    videos = negative_videos + positive_videos
    labels = [0]*len(negative_videos) + [1]*len(positive_videos)
    combined = list(zip(videos, labels))
    random.shuffle(combined)
    videos[:], labels[:] = zip(*combined)
    
    # Calculate split indices
    total = len(videos)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)
    
    # Split data
    train_videos = videos[:train_end]
    train_labels = labels[:train_end]
    val_videos = videos[train_end:val_end]
    val_labels = labels[train_end:val_end]
    test_videos = videos[val_end:]
    test_labels = labels[val_end:]
    
    return (train_videos, train_labels), (val_videos, val_labels), (test_videos, test_labels)


In [7]:
# Violence dataset
(violence_train_videos, violence_train_labels), \
(violence_val_videos, violence_val_labels), \
(violence_test_videos, violence_test_labels) = split_data(violence_negative_videos, violence_positive_videos)

# TikTok dataset
(tiktok_train_videos, tiktok_train_labels), \
(tiktok_val_videos, tiktok_val_labels), \
(tiktok_test_videos, tiktok_test_labels) = split_data(tiktok_negative_videos, tiktok_positive_videos)


In [8]:

# Convert labels to numpy arrays and one-hot encode them if necessary
def prepare_labels(labels):
    return np.array(labels)

# Violence dataset generators
violence_train_labels_np = prepare_labels(violence_train_labels)
violence_val_labels_np = prepare_labels(violence_val_labels)
violence_test_labels_np = prepare_labels(violence_test_labels)

violence_train_generator = VideoDataGenerator(violence_train_videos, violence_train_labels_np, batch_size=batch_size, shuffle=True, augment=True)
violence_val_generator = VideoDataGenerator(violence_val_videos, violence_val_labels_np, batch_size=batch_size, shuffle=False)
violence_test_generator = VideoDataGenerator(violence_test_videos, violence_test_labels_np, batch_size=batch_size, shuffle=False)

In [9]:

# TikTok dataset generators
tiktok_train_labels_np = prepare_labels(tiktok_train_labels)
tiktok_val_labels_np = prepare_labels(tiktok_val_labels)
tiktok_test_labels_np = prepare_labels(tiktok_test_labels)

tiktok_train_generator = VideoDataGenerator(tiktok_train_videos, tiktok_train_labels_np, batch_size=batch_size, shuffle=True, augment=True)
tiktok_val_generator = VideoDataGenerator(tiktok_val_videos, tiktok_val_labels_np, batch_size=batch_size, shuffle=False)
tiktok_test_generator = VideoDataGenerator(tiktok_test_videos, tiktok_test_labels_np, batch_size=batch_size, shuffle=False)


In [10]:

# Load the model
model = load_c3d_model(input_shape=input_shape, feature_dim=feature_dim)


In [11]:
# Modify the model to output class probabilities


# Freeze the base model if desired
for layer in model.layers:
    layer.trainable = False

# Add classification layer
features = model.output
outputs = layers.Dense(num_classes, activation='softmax')(features)
classification_model = models.Model(inputs=model.input, outputs=outputs)

# Compile the model
classification_model.compile(
    loss=losses.SparseCategoricalCrossentropy(),
    optimizer=optimizers.Adam(learning_rate=1e-4),
    metrics=[metrics.SparseCategoricalAccuracy()]
)


In [12]:
# Train on Violence dataset
history_violence = classification_model.fit(
    violence_train_generator,
    validation_data=violence_val_generator,
    epochs=num_epochs
)


Epoch 1/5
27/27 [==============================] - 30s 697ms/step - loss: 0.6936 - sparse_categorical_accuracy: 0.4630 - val_loss: 0.6934 - val_sparse_categorical_accuracy: 0.4643
Epoch 2/5
27/27 [==============================] - 19s 701ms/step - loss: 0.6933 - sparse_categorical_accuracy: 0.4815 - val_loss: 0.6935 - val_sparse_categorical_accuracy: 0.4643
Epoch 3/5
27/27 [==============================] - 19s 700ms/step - loss: 0.6935 - sparse_categorical_accuracy: 0.5046 - val_loss: 0.6936 - val_sparse_categorical_accuracy: 0.4643
Epoch 4/5
27/27 [==============================] - 19s 700ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.4491 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.4643
Epoch 5/5
27/27 [==============================] - 19s 715ms/step - loss: 0.6933 - sparse_categorical_accuracy: 0.4954 - val_loss: 0.6934 - val_sparse_categorical_accuracy: 0.4643


In [13]:
# Evaluate on Violence test set
base_c3d_results_violence = classification_model.evaluate(violence_test_generator)
print(f"Violence Dataset - Test Loss: {base_c3d_results_violence[0]}, Test Accuracy: {base_c3d_results_violence[1]}")


15/15 [==============================] - 8s 567ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.4667
Violence Dataset - Test Loss: 0.6932104825973511, Test Accuracy: 0.46666666865348816


In [14]:
history_tiktok = classification_model.fit(
    tiktok_train_generator,
    validation_data=tiktok_val_generator,
    epochs=num_epochs
)

Epoch 1/5
27/27 [==============================] - 61s 2s/step - loss: 0.6937 - sparse_categorical_accuracy: 0.4815 - val_loss: 0.6926 - val_sparse_categorical_accuracy: 0.6071
Epoch 2/5
27/27 [==============================] - 58s 2s/step - loss: 0.6936 - sparse_categorical_accuracy: 0.4537 - val_loss: 0.6930 - val_sparse_categorical_accuracy: 0.6429
Epoch 3/5
27/27 [==============================] - 58s 2s/step - loss: 0.6928 - sparse_categorical_accuracy: 0.4815 - val_loss: 0.6932 - val_sparse_categorical_accuracy: 0.5357
Epoch 4/5
27/27 [==============================] - 58s 2s/step - loss: 0.6922 - sparse_categorical_accuracy: 0.5324 - val_loss: 0.6934 - val_sparse_categorical_accuracy: 0.3214
Epoch 5/5
27/27 [==============================] - 58s 2s/step - loss: 0.6927 - sparse_categorical_accuracy: 0.4815 - val_loss: 0.6937 - val_sparse_categorical_accuracy: 0.3929


In [15]:

# Evaluate on TikTok test set
base_c3d_results_tiktok = classification_model.evaluate(tiktok_test_generator)
print(f"TikTok Dataset - Test Loss: {base_c3d_results_tiktok[0]}, Test Accuracy: {base_c3d_results_tiktok[1]}")

15/15 [==============================] - 27s 2s/step - loss: 0.6933 - sparse_categorical_accuracy: 0.4500
TikTok Dataset - Test Loss: 0.6933118104934692, Test Accuracy: 0.44999998807907104


## MSupCL implementation

In [16]:
# Combine training data from both datasets
combined_train_videos = violence_train_videos + tiktok_train_videos
combined_train_labels = violence_train_labels_np.tolist() + tiktok_train_labels_np.tolist()

# Create a combined data generator
combined_train_generator = VideoDataGenerator(combined_train_videos, combined_train_labels, batch_size=batch_size, shuffle=True, augment=True)


In [17]:


# Create separate generators for violence and tiktok datasets
violence_train_generator_no_aug = VideoDataGenerator(violence_train_videos, violence_train_labels_np, batch_size=batch_size, shuffle=True, augment=False)
tiktok_train_generator_no_aug = VideoDataGenerator(tiktok_train_videos, tiktok_train_labels_np, batch_size=batch_size, shuffle=True, augment=False)

# Create paired data generator
paired_train_generator = PairedDataGenerator(violence_train_generator_no_aug, tiktok_train_generator_no_aug)




In [18]:
# Load the model
msupcl_model = load_c3d_model(input_shape=input_shape)

# Train the model
train_msupcl_model(msupcl_model, paired_train_generator, epochs=num_epochs)


Epoch 1/5
Training Loss: 0.6468
Epoch 2/5
Training Loss: 0.6652
Epoch 3/5
Training Loss: 0.6503
Epoch 4/5
Training Loss: 0.6559
Epoch 5/5
Training Loss: 0.6560


In [20]:
msupcl_c3d_result_violence, msupcl_c3d_result_tiktok = linear_evaluation(msupcl_model, combined_train_generator,violence_test_generator, tiktok_test_generator)


Epoch 1/5
55/55 [==============================] - 72s 1s/step - loss: 4.3188 - accuracy: 0.5114 - val_loss: 1.0601 - val_accuracy: 0.4833
Epoch 2/5
55/55 [==============================] - 71s 1s/step - loss: 4.7202 - accuracy: 0.4659 - val_loss: 1.6083 - val_accuracy: 0.4833
Epoch 3/5
55/55 [==============================] - 73s 1s/step - loss: 3.7220 - accuracy: 0.5273 - val_loss: 1.9219 - val_accuracy: 0.4833
Epoch 4/5
55/55 [==============================] - 72s 1s/step - loss: 3.9980 - accuracy: 0.5386 - val_loss: 1.5273 - val_accuracy: 0.4833
Epoch 5/5
55/55 [==============================] - 71s 1s/step - loss: 4.1318 - accuracy: 0.4932 - val_loss: 1.3125 - val_accuracy: 0.4833
Evaluating on Violence Test Set:
15/15 [==============================] - 8s 548ms/step - loss: 1.2762 - accuracy: 0.4833
Violence Test Loss: 1.2762328386306763, Test Accuracy: 0.4833333194255829
Evaluating on TikTok Test Set:
15/15 [==============================] - 26s 2s/step - loss: 1.4197 - accuracy

## R2+1d_18 model

In [10]:
from model_train_r2plus1d_18 import load_r2plus1d_model



In [11]:
base_model = load_r2plus1d_model(input_shape=input_shape, feature_dim=feature_dim, include_top=False)

In [12]:
for layer in base_model.layers:
    layer.trainable = False
    
features = model.output
outputs = layers.Dense(num_classes, activation='softmax')(features)
classification_model = models.Model(inputs=model.input, outputs=outputs)

classification_model.compile(
    loss=losses.SparseCategoricalCrossentropy(),
    optimizer=optimizers.Adam(learning_rate=1e-4),
    metrics=[metrics.SparseCategoricalAccuracy()]
)

NameError: name 'model' is not defined

In [24]:
# Train on Violence dataset
history_violence = classification_model.fit(
    violence_train_generator,
    validation_data=violence_val_generator,
    epochs=num_epochs
)

Epoch 1/5
27/27 [==============================] - 20s 725ms/step - loss: 0.6939 - sparse_categorical_accuracy: 0.4815 - val_loss: 0.6937 - val_sparse_categorical_accuracy: 0.4643
Epoch 2/5
27/27 [==============================] - 19s 694ms/step - loss: 0.6943 - sparse_categorical_accuracy: 0.4769 - val_loss: 0.6939 - val_sparse_categorical_accuracy: 0.4643
Epoch 3/5
27/27 [==============================] - 19s 695ms/step - loss: 0.6936 - sparse_categorical_accuracy: 0.4815 - val_loss: 0.6938 - val_sparse_categorical_accuracy: 0.4643
Epoch 4/5
27/27 [==============================] - 19s 728ms/step - loss: 0.6938 - sparse_categorical_accuracy: 0.4537 - val_loss: 0.6935 - val_sparse_categorical_accuracy: 0.4643
Epoch 5/5
27/27 [==============================] - 19s 720ms/step - loss: 0.6935 - sparse_categorical_accuracy: 0.4676 - val_loss: 0.6938 - val_sparse_categorical_accuracy: 0.4643


In [25]:
# Evaluate on Violence test set
base_r2plus1d_results_violence = classification_model.evaluate(violence_test_generator)
print(f"Violence Dataset - Test Loss: {base_r2plus1d_results_violence[0]}, Test Accuracy: {base_r2plus1d_results_violence[1]}")

15/15 [==============================] - 8s 568ms/step - loss: 0.6931 - sparse_categorical_accuracy: 0.5167
Violence Dataset - Test Loss: 0.6931416392326355, Test Accuracy: 0.5166666507720947


In [26]:
history_tiktok = classification_model.fit(
    tiktok_train_generator,
    validation_data=tiktok_val_generator,
    epochs=num_epochs
)

Epoch 1/5
27/27 [==============================] - 61s 2s/step - loss: 0.6952 - sparse_categorical_accuracy: 0.4028 - val_loss: 0.6923 - val_sparse_categorical_accuracy: 0.6071
Epoch 2/5
27/27 [==============================] - 59s 2s/step - loss: 0.6948 - sparse_categorical_accuracy: 0.3935 - val_loss: 0.6929 - val_sparse_categorical_accuracy: 0.5714
Epoch 3/5
27/27 [==============================] - 58s 2s/step - loss: 0.6933 - sparse_categorical_accuracy: 0.5185 - val_loss: 0.6932 - val_sparse_categorical_accuracy: 0.3571
Epoch 4/5
27/27 [==============================] - 59s 2s/step - loss: 0.6941 - sparse_categorical_accuracy: 0.4306 - val_loss: 0.6936 - val_sparse_categorical_accuracy: 0.3929
Epoch 5/5
27/27 [==============================] - 58s 2s/step - loss: 0.6929 - sparse_categorical_accuracy: 0.4630 - val_loss: 0.6938 - val_sparse_categorical_accuracy: 0.3929


In [27]:
# Evaluate on TikTok test set
base_r2plus1d_results_tiktok = classification_model.evaluate(tiktok_test_generator)
print(f"TikTok Dataset - Test Loss: {base_r2plus1d_results_tiktok[0]}, Test Accuracy: {base_r2plus1d_results_tiktok[1]}")

15/15 [==============================] - 26s 2s/step - loss: 0.6935 - sparse_categorical_accuracy: 0.4500
TikTok Dataset - Test Loss: 0.6934583187103271, Test Accuracy: 0.44999998807907104


In [28]:

msupcl_model = load_r2plus1d_model(input_shape=input_shape, feature_dim=feature_dim, include_top=False)

In [29]:


violence_train_generator_no_aug = VideoDataGenerator(violence_train_videos, violence_train_labels_np, batch_size=batch_size, shuffle=True, augment=False)
tiktok_train_generator_no_aug = VideoDataGenerator(tiktok_train_videos, tiktok_train_labels_np, batch_size=batch_size, shuffle=True, augment=False)
paired_train_generator = PairedDataGenerator(violence_train_generator_no_aug, tiktok_train_generator_no_aug)

In [30]:
train_msupcl_model(msupcl_model, paired_train_generator, epochs=num_epochs)

Epoch 1/5
Training Loss: 0.7207
Epoch 2/5
Training Loss: 0.7167
Epoch 3/5
Training Loss: 0.7179
Epoch 4/5
Training Loss: 0.7197
Epoch 5/5
Training Loss: 0.7160


In [31]:
msupcl_r2plus1d_result_violence, msupcl_r2plus1d_result_tiktok = linear_evaluation(msupcl_model, combined_train_generator, violence_test_generator,tiktok_test_generator, num_classes=2)

Epoch 1/5
55/55 [==============================] - 73s 1s/step - loss: 0.6928 - accuracy: 0.5409 - val_loss: 0.6946 - val_accuracy: 0.5167
Epoch 2/5
55/55 [==============================] - 71s 1s/step - loss: 0.6986 - accuracy: 0.5364 - val_loss: 0.6861 - val_accuracy: 0.5833
Epoch 3/5
55/55 [==============================] - 72s 1s/step - loss: 0.6925 - accuracy: 0.5114 - val_loss: 0.6938 - val_accuracy: 0.5000
Epoch 4/5
55/55 [==============================] - 71s 1s/step - loss: 0.6948 - accuracy: 0.5250 - val_loss: 0.6948 - val_accuracy: 0.5000
Epoch 5/5
55/55 [==============================] - 72s 1s/step - loss: 0.6963 - accuracy: 0.5045 - val_loss: 0.6878 - val_accuracy: 0.5500
Evaluating on Violence Test Set:
15/15 [==============================] - 8s 547ms/step - loss: 0.6956 - accuracy: 0.5167
Violence Test Loss: 0.6956499218940735, Test Accuracy: 0.5166666507720947
Evaluating on TikTok Test Set:
15/15 [==============================] - 26s 2s/step - loss: 0.6969 - accuracy

# SSCL 

In [13]:
from data_uniform_sscl import SSCLVideoDataGenerator
from model_train import train_simclr_model, linear_evaluation_sscl,load_c3d_sscl_model

violence_train_sscl_generator = SSCLVideoDataGenerator(violence_train_videos,violence_train_labels_np,batch_size=batch_size, shuffle=True, augment=True)

violence_val_single_sscl_generator = SSCLVideoDataGenerator(violence_val_videos,violence_val_labels_np,batch_size=batch_size, shuffle=True, augment=True, double_view=False)


violence_test_sscl_generator = SSCLVideoDataGenerator(
    violence_test_videos,
    violence_test_labels_np,
    batch_size=batch_size,
    shuffle=False,
    augment=False, 
    double_view=False
)

tiktok_train_sscl_generator = SSCLVideoDataGenerator(tiktok_train_videos,tiktok_train_labels_np,batch_size=batch_size, shuffle=True, augment=True)

tiktok_val_single_sscl_generator = SSCLVideoDataGenerator(tiktok_val_videos,tiktok_val_labels_np,batch_size=batch_size, shuffle=True, augment=True, double_view=False)

tiktok_test_sscl_generator = SSCLVideoDataGenerator(
    tiktok_test_videos,
    tiktok_test_labels_np,
    batch_size=batch_size,
    shuffle=False,
    augment=False, 
    double_view=False
)

model = load_c3d_sscl_model(input_shape=input_shape, feature_dim=feature_dim)

train_simclr_model(model, violence_train_sscl_generator, epochs=num_epochs, temperature=0.5)

Epoch 1/1
Epoch 1, Loss: 1.9405


In [16]:
violence_train_single_sscl_generator = SSCLVideoDataGenerator(violence_train_videos,violence_train_labels_np,batch_size=batch_size, shuffle=True, augment=True, double_view=False)

sscl_c3d_result_violence = linear_evaluation_sscl(model, violence_train_single_sscl_generator, violence_val_single_sscl_generator,violence_test_sscl_generator, num_classes=2)

Epoch 1/5
27/27 [==============================] - 77s 3s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5167
Epoch 2/5
27/27 [==============================] - 75s 3s/step - loss: 0.6932 - accuracy: 0.4907 - val_loss: 0.6932 - val_accuracy: 0.5167
Epoch 3/5
27/27 [==============================] - 74s 3s/step - loss: 0.6933 - accuracy: 0.5185 - val_loss: 0.6932 - val_accuracy: 0.5167
Epoch 4/5
27/27 [==============================] - 78s 3s/step - loss: 0.6930 - accuracy: 0.5185 - val_loss: 0.6932 - val_accuracy: 0.5167
Epoch 5/5
15/15 [==============================] - 28s 2s/step - loss: 0.6933 - accuracy: 0.5167
Test Loss: 0.6933114528656006, Test Accuracy: 0.5166666507720947


In [ ]:
tiktok_train_single_sscl_generator = SSCLVideoDataGenerator(tiktok_train_videos,tiktok_train_labels_np,batch_size=batch_size, shuffle=True, augment=True, double_view=False)


train_simclr_model(model, tiktok_train_sscl_generator, epochs=num_epochs, temperature=0.5)
sscl_c3d_result_tiktok = linear_evaluation_sscl(model, tiktok_train_single_sscl_generator,tiktok_val_single_sscl_generator,tiktok_test_sscl_generator, num_classes=2)